In [ ]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [ ]:
import torch

from src.dvst.datasets import PanopticDataset

In [3]:
torch.__version__

'2.6.0+cu126'

In [ ]:
d = PanopticDataset('../res/tmp/scenes')

In [ ]:
d.scenes

[['170307_dance3',
  [{'K': [[1633.51, 0, 947.691], [0, 1629.51, 571.838], [0, 0, 1]],
    'R': [[-0.5295805751, -0.01150942367, 0.8481815535],
     [0.6366490414, 0.6553762361, 0.4063988032],
     [-0.56055545, 0.7552148849, -0.3397470605]],
    'filename': '../res/tmp/scenes/170307_dance3/hd_00_04.mp4',
    'fps': 29.97,
    'shape': [9211, 3, 1080, 1920],
    't': [[-5.652077537], [81.64650446], [378.2933504]],
    'type': 'hd',
    'url': 'http://domedb.perception.cs.cmu.edu/webdata/dataset/170307_dance3/videos/hd_shared_crf20/hd_00_04.mp4'},
   {'K': [[1861.42, 0, 935.575], [0, 1856.72, 553.241], [0, 0, 1]],
    'R': [[0.979300228, 0.02227476313, 0.2011837426],
     [0.03031339467, 0.9665779971, -0.2545742989],
     [-0.2001303612, 0.2554032312, 0.9458948293]],
    'filename': '../res/tmp/scenes/170307_dance3/hd_00_27.mp4',
    'fps': 29.97,
    'shape': [9211, 3, 1080, 1920],
    't': [[-0.5190184928], [123.2894969], [311.5216895]],
    'type': 'hd',
    'url': 'http://domedb.per

In [6]:
v = d.__getitem__(0)
v2 = d.__getitem__(1)
v, v2

([(<torchcodec.decoders._video_decoder.VideoDecoder at 0x7f16fbe15220>,
   tensor([[1.6335e+03, 0.0000e+00, 9.4769e+02],
           [0.0000e+00, 1.6295e+03, 5.7184e+02],
           [0.0000e+00, 0.0000e+00, 1.0000e+00]]),
   tensor([[-0.5296, -0.0115,  0.8482],
           [ 0.6366,  0.6554,  0.4064],
           [-0.5606,  0.7552, -0.3397]]),
   tensor([[ -5.6521],
           [ 81.6465],
           [378.2934]]),
   29.97,
   torch.Size([9211, 3, 1080, 1920])),
  (<torchcodec.decoders._video_decoder.VideoDecoder at 0x7f175f0d3da0>,
   tensor([[1.8614e+03, 0.0000e+00, 9.3558e+02],
           [0.0000e+00, 1.8567e+03, 5.5324e+02],
           [0.0000e+00, 0.0000e+00, 1.0000e+00]]),
   tensor([[ 0.9793,  0.0223,  0.2012],
           [ 0.0303,  0.9666, -0.2546],
           [-0.2001,  0.2554,  0.9459]]),
   tensor([[ -0.5190],
           [123.2895],
           [311.5217]]),
   29.97,
   torch.Size([9211, 3, 1080, 1920])),
  (<torchcodec.decoders._video_decoder.VideoDecoder at 0x7f186daa6210>,
  

In [ ]:
v[0][0]._num_frames, v2[0][0]._num_frames

(9211, 11598)

In [8]:
v[0][0][-1].shape, v2[0][0][-1].shape

(torch.Size([3, 1080, 1920]), torch.Size([3, 1080, 1920]))

In [ ]:
def compute_origin(v):
    K, R, t = v[1], v[2], v[3]
    o = -R.T @ t
    # d = R.T @ K.inverse() @ x
    return o


In [10]:
a = [compute_origin(i) for i in v]
a2 = [compute_origin(i) for i in v2]
a, a2

# Scenes:
# 170307_dance3, hd videos 4, 27, 24, 0
# 170404_haggling_a2, hd videos 7, 0, 14, 5

([tensor([[ 157.0810],
          [-339.2670],
          [ 100.1370]]),
  tensor([[  59.1159],
          [-198.7210],
          [-263.1760]]),
  tensor([[-255.5030],
          [-119.3360],
          [  98.2647]]),
  tensor([[-272.2060],
          [-151.6980],
          [ -21.8477]])],
 [tensor([[  75.8885],
          [-312.0810],
          [ 197.1940]]),
  tensor([[-271.4690],
          [-151.7310],
          [ -21.6585]]),
  tensor([[ 151.9930],
          [-120.3060],
          [-232.8210]]),
  tensor([[-105.1600],
          [-152.5280],
          [ 254.7600]])])

TODO I ACTUALLY CHANGED BACK SINCE THIS (x, -y, -z) (right, down, forward) IS THE CONVENTION USED BY COLMAP AND ALSO USED IN MOST DATASETS
SO I WILL ACTUALLY USE THAT CONVENTION INSTEAD
in that case, for both panoptic and plenoptic, the matrix $Q$ will be the identity.

From this, looking forward from the door of the dome towards the other side of it, we can find the conventions for the axes:

- x is the right axis, with the origin in the dome center;
- y is the down axis, with the origin in the ground;
- z is the forward axis, with the origin in the dome center.

Then, in order to map that space to the convention that we will use, we should use the following matrix:

$$
Q = Q^{-1} =
\begin{bmatrix}
1 & 0 & 0 \\
0 & -1 & 0 \\
0 & 0 & -1
\end{bmatrix}
$$


In [13]:
def compute_origin_transformed(v):
    Q = torch.tensor([[1.0, 0, 0], [0, -1, 0], [0, 0, -1]])
    return Q @ compute_origin(v)


In [15]:
a = [compute_origin_transformed(i) for i in v]
a2 = [compute_origin_transformed(i) for i in v2]
a, a2

([tensor([[ 157.0810],
          [ 339.2670],
          [-100.1370]]),
  tensor([[ 59.1159],
          [198.7210],
          [263.1760]]),
  tensor([[-255.5030],
          [ 119.3360],
          [ -98.2647]]),
  tensor([[-272.2060],
          [ 151.6980],
          [  21.8477]])],
 [tensor([[  75.8885],
          [ 312.0810],
          [-197.1940]]),
  tensor([[-271.4690],
          [ 151.7310],
          [  21.6585]]),
  tensor([[151.9930],
          [120.3060],
          [232.8210]]),
  tensor([[-105.1600],
          [ 152.5280],
          [-254.7600]])])